In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Evaluate and Select Model Generation Settings | Rapid Evaluation SDK Tutorial 

 <table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluate_gemini_model_settings_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluate_gemini_model_settings_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/evaluate_gemini_model_settings_rapid_evaluation_sdk.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluate_gemini_model_settings_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Jason Dai](https://github.com/jsondai) |

## Overview

Evaluate and Select Model Generation Settings: 

* Adjust temperature and other model generation configurations of Gemini on a summarization task and compare the evaluation results from different generation settings on quality, fluency, safety, and verbosity, etc.


## Getting Started

## Install Vertex AI SDK for Rapid Evaluation

In [32]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[rapid_evaluation]==1.47
%pip install --quiet --upgrade nest_asyncio

### Authenticate your notebook environment (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [33]:
# General
import inspect
from uuid import uuid4
from google.colab import auth
from IPython.display import display, Markdown, HTML
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
import nest_asyncio
import warnings
import random
import string
import os

# Main
import vertexai
from vertexai.preview.evaluation import (
    EvalTask,
    PromptTemplate,
    CustomMetric,
    make_metric,
)
import pandas as pd
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, HarmCategory, HarmBlockThreshold

### Library settings

In [34]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
nest_asyncio.apply()
warnings.filterwarnings("ignore")

### Helper functions

In [36]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specifed length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown(f"### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["instruction", "context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))


def plot_radar_plot(eval_results, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )

    fig.show()


def plot_bar_plot(eval_results, metrics=None):
    fig = go.Figure()
    data = []

    for eval_result in eval_results:
        title, summary_metrics, _ = eval_result
        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()


def print_aggregated_metrics(job):
    """Print AutoMetrics"""

    rougeLSum = round(job.rougeLSum, 3) * 100
    display(
        HTML(
            f"<h3>The {rougeLSum}% of the reference summary is represented by LLM when considering the longest common subsequence (LCS) of words.</h3>"
        )
    )


def print_autosxs_judgments(df, n=3):
    """Print AutoSxS judgments in the notebook"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)

    for index, row in df.iterrows():
        if row["confidence"] >= 0.5:
            display(
                HTML(
                    f"<h2>Document:</h2> <div style='{style}'>{row['id_columns']['document']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response A:</h2> <div style='{style}'>{row['response_a']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response B:</h2> <div style='{style}'>{row['response_b']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Explanation:</h2> <div style='{style}'>{row['explanation']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Confidence score:</h2> <div style='{style}'>{row['confidence']}</div>"
                )
            )
            display(HTML("<hr>"))


def print_autosxs_win_metrics(scores):
    """Print AutoSxS aggregated metrics"""

    score_b = round(scores["autosxs_model_b_win_rate"] * 100)
    display(
        HTML(
            f"<h3>AutoSxS Autorater prefers {score_b}% of time Model B over Model A </h3>"
        )
    )

## Evaluate and Select Model Generation Settings


### Define an evaluation dataset

In [37]:
instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]


eval_dataset = pd.DataFrame(
    {
        "context": context,
        "instruction": [instruction] * len(context),
        "reference": reference,
    }
)

### Design text prompt template

For more task-specific prompt guidance, see https://cloud.google.com/vertex-ai/generative-ai/docs/text/text-prompts.

In [38]:
prompt_template = PromptTemplate("{instruction}. Article: {context}. Summary:")

### Define metrics

In [39]:
metrics = [
    "rouge_1",
    "rouge_l_sum",
    "bleu",
    "fluency",
    "coherence",
    "safety",
    "groundedness",
    "summarization_quality",
    "summarization_verbosity",
    "summarization_helpfulness",
]

### Define EvalTask & Experiment

In [40]:
experiment_name = "eval-model-generation-settings"  # @param {type:"string"}

summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=experiment_name,
)

### Evaluate Gemini Model with different Generation Settings

In [41]:
generation_config_1 = {
    "max_output_tokens": 64,
    "temperature": 0.1,
}
generation_config_2 = {
    "max_output_tokens": 128,
    "temperature": 0.3,
}
generation_config_3 = {
    "max_output_tokens": 256,
    "temperature": 0.8,
}

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

gemini_1 = GenerativeModel(
    "gemini-pro", generation_config=generation_config_1, safety_settings=safety_settings
)
gemini_2 = GenerativeModel(
    "gemini-pro", generation_config=generation_config_2, safety_settings=safety_settings
)
gemini_3 = GenerativeModel(
    "gemini-pro", generation_config=generation_config_3, safety_settings=safety_settings
)

In [42]:
models = {
    "gemini-setting-1": gemini_1,
    "gemini-setting-2": gemini_2,
    "gemini-setting-3": gemini_3,
}

In [43]:
eval_results = []
run_id = generate_uuid()

for _, (model_name, model) in tqdm(
    enumerate(zip(models.keys(), models.values())), total=len(models.keys())
):
    experiment_run_name = f"eval-{model_name}-{run_id}"

    eval_result = summarization_eval_task.evaluate(
        model=model,
        prompt_template=prompt_template,
        experiment_run_name=experiment_run_name,
    )

    eval_results.append(
        (f"Model {model_name}", eval_result.summary_metrics, eval_result.metrics_table)
    )

  0%|          | 0/3 [00:00<?, ?it/s]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-1-e9b8ckyo to Experiment: eval-model-generation-settings
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': PromptTemplate('{instruction}. Article: {context}. Summary:'), 'model_name': 'publishers/google/models/gemini-pro', 'max_output_tokens': 64, 'temperature': 0.1, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


 33%|███▎      | 1/3 [00:45<01:30, 45.16s/it]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-2-e9b8ckyo to Experiment: eval-model-generation-settings
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': PromptTemplate('{instruction}. Article: {context}. Summary:'), 'model_name': 'publishers/google/models/gemini-pro', 'max_output_tokens': 128, 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


 67%|██████▋   | 2/3 [01:16<00:36, 36.77s/it]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-3-e9b8ckyo to Experiment: eval-model-generation-settings
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': PromptTemplate('{instruction}. Article: {context}. Summary:'), 'model_name': 'publishers/google/models/gemini-pro', 'max_output_tokens': 256, 'temperature': 0.8, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


100%|██████████| 3/3 [02:12<00:00, 44.21s/it]


### Evaluation Report

In [44]:
for eval_result in eval_results:
    display_eval_report(eval_result)

## Model gemini-setting-1

### Summary Metrics

,row_count,rouge_1/mean,rouge_1/std,rouge_l_sum/mean,rouge_l_sum/std,bleu/mean,bleu/std,fluency/mean,fluency/std,coherence/mean,...,safety/mean,safety/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,summarization_verbosity/mean,summarization_verbosity/std,summarization_helpfulness/mean,summarization_helpfulness/std
0,10.0,0.393383,0.15324,0.357452,0.154416,0.096261,0.066743,3.6,0.843274,1.3,...,1.0,0.0,0.9,0.316228,5.0,0.0,0.0,0.0,3.6,0.516398


### Report Metrics

,context,instruction,reference,completed_prompt,response,rouge_1,rouge_l_sum,bleu,fluency/explanation,fluency/confidence,...,groundedness,summarization_quality/explanation,summarization_quality/confidence,summarization_quality,summarization_verbosity/explanation,summarization_verbosity/confidence,summarization_verbosity,summarization_helpfulness/explanation,summarization_helpfulness/confidence,summarization_helpfulness
0,The kitchen is a place where ingredients are t...,Summarize the following article,Cooking is a multi-sensory experience of trans...,Summarize the following article. Article: The ...,The kitchen is a culinary haven where ingredie...,0.363636,0.303030,0.051182,"The response has some grammatical errors, such...",0.9,...,0.0,The summary in response follows instructions t...,0.8,5.0,The response is concise and effectively convey...,1.0,0.0,The response captures the main idea of the ori...,0.8,4.0
1,"Mise en place, the practice of preparing ingre...",Summarize the following article,Preparing ingredients beforehand (mise en plac...,Summarize the following article. Article: Mise...,"Mise en place, the preparation of ingredients ...",0.333333,0.266667,0.039566,"The response has no grammatical errors, and th...",0.6,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and summarizes the mai...,1.0,0.0,The response captures the main point of the ar...,0.9,4.0
2,The Maillard reaction is the reason behind the...,Summarize the following article,The Maillard reaction creates delicious flavor...,Summarize the following article. Article: The ...,The Maillard reaction is a chemical process th...,0.363636,0.303030,0.072382,The sentence has no grammatical errors and is ...,0.6,...,1.0,The response follows instructions to summarize...,1.0,5.0,The response is very concise and provides only...,1.0,0.0,The summary captures the main idea of the Mail...,1.0,4.0
3,Salt is an essential seasoning that enhances a...,Summarize the following article,Salt is a versatile seasoning that improves th...,Summarize the following article. Article: Salt...,Salt is a crucial seasoning that elevates the ...,0.666667,0.666667,0.172007,The response has some grammatical errors and a...,0.8,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and provides the neces...,1.0,0.0,The response captures the main idea of the ori...,1.0,3.0
4,A sharp knife allows for safe and precise cuts...,Summarize the following article,Sharp knives are essential for safe and effici...,Summarize the following article. Article: A sh...,A sharp knife is essential for safe and accura...,0.454545,0.454545,0.204480,"The response has no grammatical errors, uses a...",0.8,...,1.0,The response follows the instruction to summar...,1.0,5.0,The response is concise and provides only the ...,1.0,0.0,The response captures the main point of the or...,1.0,3.0
5,Emulsification is the technique used to blend ...,Summarize the following article,Emulsification allows you to create creamy sau...,Summarize the following article. Article: Emul...,Emulsification is a technique that combines no...,0.148148,0.148148,0.023901,The response has no grammatical errors and dem...,0.8,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and provides a clear d...,1.0,0.0,The response provides a concise and accurate s...,0.9,4.0
6,Tasting your food as you cook allows you to cu...,Summarize the following article,Tasting as you cook helps you achieve the perf...,Summarize the following article. Article: Tast...,Tasting food while cooking enables customizati...,0.285714,0.214286,0.054012,"The response has some grammatical errors, such...",0.5,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and provides all the n...,1.0,0.0,The response captures the main idea of the art...,0.9,4.0
7,"Patience is key for success in cooking, allowi...",Summarize the following article,

...

## Model gemini-setting-3

### Summary Metrics

,row_count,rouge_1/mean,rouge_1/std,rouge_l_sum/mean,rouge_l_sum/std,bleu/mean,bleu/std,fluency/mean,fluency/std,coherence/mean,...,safety/mean,safety/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,summarization_verbosity/mean,summarization_verbosity/std,summarization_helpfulness/mean,summarization_helpfulness/std
0,10.0,0.350246,0.136322,0.317858,0.123657,0.070518,0.037736,3.6,0.843274,1.3,...,1.0,0.0,0.8,0.421637,5.0,0.0,0.0,0.0,3.4,0.516398


### Report Metrics

,context,instruction,reference,completed_prompt,response,rouge_1,rouge_l_sum,bleu,fluency/explanation,fluency/confidence,...,groundedness,summarization_quality/explanation,summarization_quality/confidence,summarization_quality,summarization_verbosity/explanation,summarization_verbosity/confidence,summarization_verbosity,summarization_helpfulness/explanation,summarization_helpfulness/confidence,summarization_helpfulness
0,The kitchen is a place where ingredients are t...,Summarize the following article,Cooking is a multi-sensory experience of trans...,Summarize the following article. Article: The ...,"In the kitchen, ingredients are magically tran...",0.240000,0.240000,0.054130,The response has no grammatical errors and use...,0.6,...,0.0,The response follows the instruction to summar...,1.0,5.0,The response is concise and captures the main ...,1.0,0.0,The response captures the main idea of the ori...,1.0,3.0
1,"Mise en place, the practice of preparing ingre...",Summarize the following article,Preparing ingredients beforehand (mise en plac...,Summarize the following article. Article: Mise...,"Mise en place, preparing ingredients in advanc...",0.480000,0.320000,0.054012,"The response has grammatical errors present, w...",1.0,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and provides the essen...,1.0,0.0,The response captures the main idea of mise en...,0.9,3.0
2,The Maillard reaction is the reason behind the...,Summarize the following article,The Maillard reaction creates delicious flavor...,Summarize the following article. Article: The ...,"The Maillard reaction, occurring during cookin...",0.461538,0.384615,0.109349,"The response has no grammatical errors, uses a...",0.5,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response is concise and provides the essen...,1.0,0.0,The response captures the main point of the ar...,0.9,4.0
3,Salt is an essential seasoning that enhances a...,Summarize the following article,Salt is a versatile seasoning that improves th...,Summarize the following article. Article: Salt...,Salt is a key ingredient that adds flavor to a...,0.444444,0.444444,0.105215,The response lacks smooth transitions between ...,0.4,...,1.0,The response follows the instruction to summar...,1.0,5.0,The response is concise and provides all the n...,0.8,0.0,The response captures the main idea that salt ...,0.6,3.0
4,A sharp knife allows for safe and precise cuts...,Summarize the following article,Sharp knives are essential for safe and effici...,Summarize the following article. Article: A sh...,Sharp knives ensure safe and accurate ingredie...,0.555556,0.555556,0.132248,The sentence has no grammatical errors and is ...,1.0,...,1.0,The response follows the instruction to summar...,1.0,5.0,The response is concise and provides all the n...,0.5,0.0,The response captures the main point of the or...,0.9,3.0
5,Emulsification is the technique used to blend ...,Summarize the following article,Emulsification allows you to create creamy sau...,Summarize the following article. Article: Emul...,Emulsification is a technique that combines un...,0.190476,0.190476,0.040654,The response lacks smooth transitions and has ...,0.5,...,1.0,The response follows the instruction to summar...,1.0,5.0,The response is concise and provides the essen...,0.8,0.0,The response captures the main idea of emulsif...,0.8,3.0
6,Tasting your food as you cook allows you to cu...,Summarize the following article,Tasting as you cook helps you achieve the perf...,Summarize the following article. Article: Tast...,Tasting food while cooking enables personaliza...,0.137931,0.137931,0.028265,"The response has some grammatical errors, such...",0.8,...,1.0,The summary in response follows the instructio...,1.0,5.0,The response is concise and summarizes the mai...,1.0,0.0,The response captures the main idea of the ori...,0.9,4.0
7,"Patience is key for success in cooking, allowi...",Summarize the following articl

In [45]:
for eval_result in eval_results:
    display_explanations(eval_result[2], metrics=["fluency"])

### Compare Eval Results

In [46]:
plot_radar_plot(
    eval_results,
    metrics=[
        f"{metric}/mean"
        for metric in [
            "fluency",
            "coherence",
            "summarization_quality",
            "summarization_helpfulness",
        ]
    ],
)
plot_bar_plot(
    eval_results,
    metrics=[f"{metric}/mean" for metric in ["rouge_1", "rouge_l_sum", "bleu"]],
)

In [47]:
summarization_eval_task.display_runs()

,experiment_name,run_name,run_type,state,param.temperature,param.HARM_CATEGORY_UNSPECIFIED,param.max_output_tokens,param.HARM_CATEGORY_HATE_SPEECH,param.HARM_CATEGORY_SEXUALLY_EXPLICIT,param.model_name,...,metric.safety/std,metric.summarization_verbosity/mean,metric.rouge_l_sum/std,metric.fluency/std,metric.safety/mean,metric.row_count,metric.bleu/std,metric.rouge_1/mean,metric.rouge_l_sum/mean,metric.summarization_verbosity/std
0,eval-model-generation-settings,eval-gemini-setting-3-e9b8ckyo,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.123657,0.843274,1.0,10.0,0.037736,0.350246,0.317858,0.0
1,eval-model-generation-settings,eval-gemini-setting-2-e9b8ckyo,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.161161,0.699206,1.0,10.0,0.055445,0.363943,0.323401,0.0
2,eval-model-generation-settings,eval-gemini-setting-1-e9b8ckyo,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.154416,0.843274,1.0,10.0,0.066743,0.393383,0.357452,0.0
3,eval-model-generation-settings,eval-gemini-setting-1-cxvvtzrz,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,eval-model-generation-settings,eval-gemini-setting-1-99d2767f,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,eval-model-generation-settings,eval-gemini-setting-1-kkv43sl4,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,eval-model-generation-settings,eval-gemini-setting-1-8qnj7u4h,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,eval-model-generation-settings,eval-gemini-setting-3-pkev0ses,system.ExperimentRun,COMPLETE,0.8,BLOCK_NONE,256.0,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-pro,...,0.0,0.0,0.058395,0.957427,1.0,4.0,0.005033,0.166133,0.115743,0.0
8,eval-model-generation-settings,eval-gemini-setting-2-pkev0ses,system.ExperimentRun,COMPLETE,0.3,BLOCK_NONE,128.0,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-pro,...,0.0,0.0,0.014356,0.957427,1.0,4.0,0.004385,0.172389,0.138749,0.0
9,eval-model-generation-settings,eval-gemini-setting-1-pkev0ses,system.ExperimentRun,COMPLETE,0.1,BLOCK_NONE,64.0,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-pro,...,0.0,0.0,0.026356,1.500000,1.0,4.0,0.003458,0.179609,0.131420,0.0
